In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [5]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [6]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,label,text
0,4 stars,"I decided to pretend that I was a well-to-do person with no cares in the world, and I took a sudden, two-day trip to Vegas on a complete whim. Decided at 10:00 am, bought my ticket at 11:00, flew out at 3:00 and by late afternoon I was in Vegas. I met my parents, and made the executive decision to not go see Viva Elvis. Instead, I took the time to look around Aria, Cosmopolitan, etc. and wondered if I could find a good cocktail. I decided on Sage, and after telling myself I wasn't under-dressed, I wandered in and sat at the bar.\n\nNow, I personally do not think I am the most beautiful person on the planet, but I have come to the conclusion that I am not bad to look at, at least on my good days. And I was wearing a cute dress, and I put my confident face on. I ordered the \""tea rose\"" (tea infused Hangar One, rose water, st. germain, lemon, and a pine nut garnish) and settled in to wait for Elvis to let out. And then, out of nowhere, lovely businessmen started talking me up and invited me to dinner.\n\nBeing a tasteful, respectfully married woman, I had to decline, but honestly the vibe, the menu, and the drinks here had me trying to find any available way to get into said dinner while remaining a respectfully married woman. Sadly, I couldn't come up with any inventive way to do that, so I missed out on dinner. I did, though, get to take in the stylish/romantic/scene-y vibe, and apparently the extremely flattering lighting. Next time I have more time, or my husband in tow, or a relaxed set of morals, I'm taking myself back here and landing myself a dinner to go with my drink."
1,3 stars,"Disclaimer: The drinks are REALLY sweet over here, but the name says it all.\n\nDespite the already ultra-sweet and slushy drinks, the beverages also come with different types of candy, like the gummy-type, and the type that will crackle and pop in your mouth. It is an interesting concept, but the drinks are more sweet than they are strong, and after 1 or 2 you may get a headache or stomachache (...or become diabetic) from all that sugar.\n\nDrinks worth ordering: Tiffany & Co., made with Raspberry Smirnoff Vodka, Blue Cara\u00e7ao and raspberry slush, topped with a Ringpop. Yes, your lips and tongue will stain blue, but who cares? Rock it! My personal favourite drink was the Sushi Cocktail, made with Vodka, Lychee Liquor and lemon slush, served with a pair of chopsticks to pick out the gummy-candies that were swimming at the bottom of the tall glass.\n\nThe decor is very funky. The bar is adorned in neon and fluorescent lights. Seats are held up by womanly legs as the stand, which can be very amusing and entertaining when observing someone from the back. And the bathrooms are fascinating - In the girls' bathroom, one can find two toilet seats that are facing each other, for the perfect \""let's pee together and gossip\"" scenario. Meanwhile, cartoons like Betty Boop are playing on the flat screen to keep others entertained while waiting. This bathroom could quite possibly be a woman's dream come true. In the boys' bathroom, not only do they get to relieve themselves, but they get to do so in to large, wide open, red-lipstick lips. Satisfaction guaranteed? I think so.\n\nIt is not a bad place per se, it can be fun and part of a great \""girls night out,\"" however there is only so much sugar that I can handle. I would recommend it as more of a short stop since the hype dies out fast."
2,2 star,This was our first (and last time) going here. We wanted a taco Tuesday. A friend recommended it to us. We sat for at least 15 with nothing. Got our drink order filled. No chips or water for us. Food was just okay. Couldn't order desert cause nobody told our server the kitchen was closed. Too many other places to go than to go through that again...
3,3 stars,"I booked a deal here for two nights for my boyfriend and me. Got a suite for $59/night plus the $25 resort fee. The deal also included $40 at their restaurant, Simon, and 20

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

show_random_elements(tokenized_datasets["train"], num_examples=1)

/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


,label,text,input_ids,token_type_ids,attention_mask
0,1 star,"I have came here several time and ive come to the conclusion that the manager sucks! I think her names Abigail or Agnus or something. \n\n1st time- she tried to charge me a restocking fee when even on the sprint website it says as long as the box is unopened and inactivated no restocking fee will be applied. the after going back and forth she tried to make it seem like she was doing me a favor.. umm its not a favor when its the company policy lady.\n\n2nd time- I had to go back because of something that got messed up the first time.. she referred me to the 800 number when in fact the 800 number said I had to go to the store that messed up because they dont have access to installment biling info. Again, she didnt know anything but promised to figure it out all call me asap... never got a call.. \n\n3rd time- went back to get the issue resolved and she had no clue and was like oh let me see if I can call someone ill call you whether we can fix it or not... \n\nIve called 2x and still waiting on that call...","[101, 146, 1138, 1338, 1303, 1317, 1159, 1105, 178, 2707, 1435, 1106, 1103, 6593, 1115, 1103, 2618, 22797, 106, 146, 1341, 1123, 2666, 17014, 1137, 138, 17653, 1361, 1137, 1380, 119, 165, 183, 165, 183, 1475, 2050, 1159, 118, 1131, 1793, 1106, 2965, 1143, 170, 1832, 5559, 1158, 7216, 1165, 1256, 1113, 1103, 12766, 3265, 1122, 1867, 1112, 1263, 1112, 1103, 2884, 1110, 8362, 15622, 3540, 1105, 22951, 1185, 1832, 5559, 1158, 7216, 1209, 1129, 3666, 119, 1103, 1170, 1280, 1171, 1105, 5275, 1131, 1793, 1106, 1294, 1122, 3166, 1176, 1131, 1108, 1833, 1143, 170, 5010, 119, 119, 15276, 1306, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [13]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=

In [15]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.230700,1.085061,0.530000
2,0.928100,0.992480,0.586000
3,0.569800,1.014207,0.616000


TrainOutput(global_step=189, training_loss=0.9556059812111829, metrics={'train_runtime': 149.7517, 'train_samples_per_second': 20.033, 'train_steps_per_second': 1.262, 'total_flos': 789354427392000.0, 'train_loss': 0.9556059812111829, 'epoch': 3.0})

In [16]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.0995454788208008,
 'eval_accuracy': 0.57,
 'eval_runtime': 1.5099,
 'eval_samples_per_second': 66.229,
 'eval_steps_per_second': 8.61,
 'epoch': 3.0}

In [17]:
#尝试使用更多训练集
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10000))

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.889100,0.995319,0.548200
2,0.683500,0.963438,0.604400
3,0.332500,1.214426,0.611500


TrainOutput(global_step=1875, training_loss=0.6844582898457845, metrics={'train_runtime': 1430.0053, 'train_samples_per_second': 20.979, 'train_steps_per_second': 1.311, 'total_flos': 7893544273920000.0, 'train_loss': 0.6844582898457845, 'epoch': 3.0})

In [19]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.4493893384933472,
 'eval_accuracy': 0.53,
 'eval_runtime': 1.2379,
 'eval_samples_per_second': 80.783,
 'eval_steps_per_second': 10.502,
 'epoch': 3.0}

In [ ]:
#增加了训练数量，acc没有升高??